In [21]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


Simply select "GPU" in the Accelerator drop-down in Notebook Settings (either through the Edit menu or the command palette at cmd/ctrl-shift-P).

In [22]:
# Data download and unzipping- a nice kaggler rehosted data on their github. to use wget from kaggle you need authetication/cookies
!wget https://raw.githubusercontent.com/AakashSudhakar/2018-data-science-bowl/master/compressed_files/stage1_test.zip -c
#!wget https://raw.githubusercontent.com/AakashSudhakar/2018-data-science-bowl/master/compressed_files/stage1_train.zip -c
!wget https://raw.githubusercontent.com/raykallen/twosnacks/master/stage1_train.zip -c

!mkdir stage1_train stage1_test

!unzip stage1_train.zip -d stage1_train/
!unzip stage1_test.zip -d stage1_test/

--2018-03-14 20:58:28--  https://raw.githubusercontent.com/AakashSudhakar/2018-data-science-bowl/master/compressed_files/stage1_test.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 416 Range Not Satisfiable

    The file is already fully retrieved; nothing to do.

--2018-03-14 20:58:29--  https://raw.githubusercontent.com/raykallen/twosnacks/master/stage1_train.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 416 Range Not Satisfiable

    The file is already fully retrieved; nothing to do.

mkdir: cannot create directory ‘stage1_train’: File exists
mkdir: cannot

In [23]:
# Downloading and intstalling keras
!pip install keras

In [0]:
import os
import random
import sys
import warnings
import numpy as np
import pandas as pd
from itertools import chain
from skimage.io import imread, imshow, imread_collection, concatenate_images
from skimage.transform import resize
from skimage.morphology import label
from keras.utils import Progbar

from keras.models import Model, load_model
from keras.layers import Input
from keras.layers.core import Dropout, Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose,Convolution2D
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras import backend as K

from skimage.filters import threshold_otsu
from IPython.display import display,clear_output,Image
from skimage.color import rgb2gray

import os
import numpy as np
import scipy.misc
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

warnings.filterwarnings('ignore', category=UserWarning, module='skimage')

# Setting seed for reproducability
seed = 42
random.seed = seed
np.random.seed = seed
smooth = 1.
epochs = 50

In [0]:
# Data Path
TRAIN_PATH = 'stage1_train/stage1_train/'
TEST_PATH = 'stage1_test/'

In [0]:
# Get train and test IDs
train_ids = next(os.walk(TRAIN_PATH))[1]
test_ids = next(os.walk(TEST_PATH))[1]

In [7]:
train_ids.remove('__MACOSX')


ValueError: ignored

In [27]:
len(train_ids)

664

In [0]:
# Function read train images and mask return as nump array
def read_train_data(IMG_WIDTH=256,IMG_HEIGHT=256,IMG_CHANNELS=3):
    X_train = np.zeros((len(train_ids), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
    X_rot = np.zeros((len(train_ids), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
    X_rot2 = np.zeros((len(train_ids), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
    X_rot3 = np.zeros((len(train_ids), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
    X_flip = np.zeros((len(train_ids), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
    Y_train = np.zeros((len(train_ids), IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool)
    Y_rot = np.zeros((len(train_ids), IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool)
    Y_rot2 = np.zeros((len(train_ids), IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool)
    Y_rot3 = np.zeros((len(train_ids), IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool)
    Y_flip = np.zeros((len(train_ids), IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool)
    print('Getting and resizing train images and masks ... ')
    sys.stdout.flush()
    '''if os.path.isfile("train_img.npy") and os.path.isfile("train_mask.npy"):
        print("Train file loaded from memory")
        X_train = np.load("train_img.npy")
        Y_train = np.load("train_mask.npy")
        return X_train,Y_train
    a = Progbar(len(train_ids))'''
    for n, id_ in enumerate(train_ids):
        path = TRAIN_PATH + id_
        img = imread(path + '/images/' + id_ + '.png')[:,:,:IMG_CHANNELS]
        img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)

        
        X_train[n] = img
        
        mask = np.zeros((IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool)
        for mask_file in next(os.walk(path + '/masks/'))[2]:
            mask_ = imread(path + '/masks/' + mask_file)
            mask_ = np.expand_dims(resize(mask_, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True), axis=-1)
            mask = np.maximum(mask, mask_)


        Y_train[n] = mask

    for n, id_ in enumerate(train_ids):
        path = TRAIN_PATH + id_
        img = imread(path + '/images/' + id_ + '.png')[:,:,:IMG_CHANNELS]
        img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
        img1 = np.rot90(img)
        img2 = np.rot90(img1)
        img3 = np.rot90(img2)
        img4 = np.flip(img, 0)

        X_rot[n] = img1
        X_rot2[n] = img2
        X_rot3[n] = img3
        X_flip[n] = img4
        
        mask = np.zeros((IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool)
        for mask_file in next(os.walk(path + '/masks/'))[2]:
            mask_ = imread(path + '/masks/' + mask_file)
            mask_ = np.expand_dims(resize(mask_, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True), axis=-1)
            mask = np.maximum(mask, mask_)
            img1 = np.rot90(mask)
            img2 = np.rot90(img1)
            img3 = np.rot90(img2)
            img4 = np.flip(mask, 0)
        

        Y_rot[n] = img1
        Y_rot2[n] = img2
        Y_rot3[n] = img3
        Y_flip[n] = img4
    
    Y_all = np.concatenate((Y_train, Y_rot, Y_rot2, Y_rot3, Y_flip), axis=0)    
    X_all = np.concatenate((X_train, X_rot, X_rot2, X_rot3, X_flip), axis=0)    
        #a.update(n)        
        
        
        
    np.save("train_img",X_all)
    np.save("train_mask",Y_all)
    return X_all,Y_all

In [0]:
# Function to read test images and return as numpy array
def read_test_data(IMG_WIDTH=256,IMG_HEIGHT=256,IMG_CHANNELS=3):
    X_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
    sizes_test = []
    print('\nGetting and resizing test images ... ')
    sys.stdout.flush()
    if os.path.isfile("test_img.npy") and os.path.isfile("test_size.npy"):
        print("Test file loaded from memory")
        X_test = np.load("test_img.npy")
        sizes_test = np.load("test_size.npy")
        return X_test,sizes_test
    b = Progbar(len(test_ids))
    for n, id_ in enumerate(test_ids):
        path = TEST_PATH + id_
        img = imread(path + '/images/' + id_ + '.png')[:,:,:IMG_CHANNELS]
        sizes_test.append([img.shape[0], img.shape[1]])
        img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
        
        X_test[n] = img
        
        b.update(n)
    np.save("test_img",X_test)
    np.save("test_size",sizes_test)
    return X_test,sizes_test

In [0]:
# Run-length encoding stolen from https://www.kaggle.com/rakhlin/fast-run-length-encoding-python
def rle_encoding(x):
    dots = np.where(x.T.flatten() == 1)[0]
    run_lengths = []
    prev = -2
    for b in dots:
        if (b>prev+1): run_lengths.extend((b + 1, 0))
        run_lengths[-1] += 1
        prev = b
    return run_lengths

def prob_to_rles(x, cutoff=0.5):
    lab_img = label(x > cutoff)
    for i in range(1, lab_img.max() + 1):
        yield rle_encoding(lab_img == i)

In [0]:
# Iterate over the test IDs and generate run-length encodings for each seperate mask identified by skimage
def mask_to_rle(preds_test_upsampled):
    new_test_ids = []
    rles = []
    for n, id_ in enumerate(test_ids):
        rle = list(prob_to_rles(preds_test_upsampled[n]))
        rles.extend(rle)
        new_test_ids.extend([id_] * len(rle))
    return new_test_ids,rles

In [0]:
# Metric functions
def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

# Loss funtion
def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)
  
# Define IoU metric
def mean_iou(y_true, y_pred):
    prec = []
    for t in np.arange(0.5, 1.0, 0.05):
        y_pred_ = tf.to_int32(y_pred > t)
        score, up_opt = tf.metrics.mean_iou(y_true, y_pred_, 2)
        K.get_session().run(tf.local_variables_initializer())
        with tf.control_dependencies([up_opt]):
            score = tf.identity(score)
        prec.append(score)
    return K.mean(K.stack(prec), axis=0)

In [0]:
def get_unet(IMG_WIDTH=256,IMG_HEIGHT=256,IMG_CHANNELS=3):
    inputs = Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
    s = Lambda(lambda x: x / 255) (inputs)
    
    c1 = Conv2D(8, (3, 3), activation='relu', padding='same') (s)
    c1 = Conv2D(8, (3, 3), activation='relu', padding='same') (c1)
    p1 = MaxPooling2D((2, 2)) (c1)

    c2 = Conv2D(16, (3, 3), activation='relu', padding='same') (p1)
    c2 = Conv2D(16, (3, 3), activation='relu', padding='same') (c2)
    p2 = MaxPooling2D((2, 2)) (c2)

    c3 = Conv2D(32, (3, 3), activation='relu', padding='same') (p2)
    c3 = Conv2D(32, (3, 3), activation='relu', padding='same') (c3)
    p3 = MaxPooling2D((2, 2)) (c3)

    c4 = Conv2D(64, (3, 3), activation='relu', padding='same') (p3)
    c4 = Conv2D(64, (3, 3), activation='relu', padding='same') (c4)
    p4 = MaxPooling2D(pool_size=(2, 2)) (c4)

    c5 = Conv2D(128, (3, 3), activation='relu', padding='same') (p4)
    c5 = Conv2D(128, (3, 3), activation='relu', padding='same') (c5)
    p5 = MaxPooling2D(pool_size=(2,2)) (c5)
    
    c5_ = Conv2D(256, (3, 3), activation='relu', padding='same') (p5)
    c5_ = Conv2D(256, (3, 3), activation='relu', padding='same') (c5_)
    
    u6_ = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same') (c5_)
    u6_ = concatenate([u6_, c5])
    c6_ = Conv2D(64, (3, 3), activation='relu', padding='same') (u6_)
    c6_ = Conv2D(64, (3, 3), activation='relu', padding='same') (c6_)
    

    u6 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same') (c5)
    u6 = concatenate([u6, c4])
    c6 = Conv2D(64, (3, 3), activation='relu', padding='same') (u6)
    c6 = Conv2D(64, (3, 3), activation='relu', padding='same') (c6)

    u7 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same') (c6)
    u7 = concatenate([u7, c3])
    c7 = Conv2D(32, (3, 3), activation='relu', padding='same') (u7)
    c7 = Conv2D(32, (3, 3), activation='relu', padding='same') (c7)

    u8 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same') (c7)
    u8 = concatenate([u8, c2])
    c8 = Conv2D(16, (3, 3), activation='relu', padding='same') (u8)
    c8 = Conv2D(16, (3, 3), activation='relu', padding='same') (c8)

    u9 = Conv2DTranspose(8, (2, 2), strides=(2, 2), padding='same') (c8)
    u9 = concatenate([u9, c1], axis=3)
    c9 = Conv2D(8, (3, 3), activation='relu', padding='same') (u9)
    c9 = Conv2D(8, (3, 3), activation='relu', padding='same') (c9)
    

    outputs = Conv2D(1, (1, 1), activation='sigmoid') (c9)

    model = Model(inputs=[inputs], outputs=[outputs])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[mean_iou])
    return model

In [34]:
# get train_data
train_img,train_mask = read_train_data()

# get test_data
test_img,test_img_sizes = read_test_data()

Getting and resizing train images and masks ... 

Getting and resizing test images ... 
Test file loaded from memory


In [35]:
len(train_mask)

3320

In [0]:
# get u_net model
u_net = get_unet()

In [0]:
#Callbacks for Patience
SG_patience=callbacks.EarlyStopping(monitor='loss', min_delta=.0001, patience=2, verbose=0, mode='auto')

In [51]:
# fit model on train_data
print("\nTraining...")
u_net.fit(train_img,train_mask,batch_size=16,epochs=epochs,callbacks=[SG_patience])


Training...
Epoch 1/50
3320/3320 [==============================] - 87s 26ms/step - loss: 0.0851 - mean_iou: 0.7175
Epoch 2/50
3320/3320 [==============================] - 88s 26ms/step - loss: 0.0805 - mean_iou: 0.7483
Epoch 3/50
1584/3320 [=============>................] - ETA: 45s - loss: 0.0779 - mean_iou: 0.7637

3320/3320 [==============================] - 88s 26ms/step - loss: 0.0763 - mean_iou: 0.7681
Epoch 4/50
3320/3320 [==============================] - 88s 26ms/step - loss: 0.0721 - mean_iou: 0.7829
Epoch 5/50
2720/3320 [=======================>......] - ETA: 15s - loss: 0.0744 - mean_iou: 0.7929

3320/3320 [==============================] - 87s 26ms/step - loss: 0.0747 - mean_iou: 0.7937
Epoch 6/50
3320/3320 [==============================] - 88s 26ms/step - loss: 0.0685 - mean_iou: 0.8022
Epoch 7/50
3056/3320 [==========================>...] - ETA: 6s - loss: 0.0671 - mean_iou: 0.8093

3320/3320 [==============================] - 88s 26ms/step - loss: 0.0672 - mean_iou: 0.8095
Epoch 8/50
3320/3320 [==============================] - 88s 26ms/step - loss: 0.0650 - mean_iou: 0.8156
Epoch 9/50
3152/3320 [===========================>..] - ETA: 4s - loss: 0.0640 - mean_iou: 0.8207

3320/3320 [==============================] - 88s 27ms/step - loss: 0.0640 - mean_iou: 0.8209
Epoch 10/50
3320/3320 [==============================] - 88s 27ms/step - loss: 0.0642 - mean_iou: 0.8252
Epoch 11/50
3152/3320 [===========================>..] - ETA: 4s - loss: 0.0624 - mean_iou: 0.8288

...

3320/3320 [==============================] - 88s 26ms/step - loss: 0.0593 - mean_iou: 0.8401
Epoch 16/50
3320/3320 [==============================] - 87s 26ms/step - loss: 0.0590 - mean_iou: 0.8423
Epoch 17/50
3152/3320 [===========================>..] - ETA: 4s - loss: 0.0587 - mean_iou: 0.8443

3320/3320 [==============================] - 88s 26ms/step - loss: 0.0586 - mean_iou: 0.8443
Epoch 18/50
3320/3320 [==============================] - 88s 26ms/step - loss: 0.0578 - mean_iou: 0.8463
Epoch 19/50
3152/3320 [===========================>..] - ETA: 4s - loss: 0.0755 - mean_iou: 0.8479

3320/3320 [==============================] - 88s 26ms/step - loss: 0.0774 - mean_iou: 0.8479
Epoch 20/50
3320/3320 [==============================] - 88s 27ms/step - loss: 0.0710 - mean_iou: 0.8479
Epoch 21/50
3152/3320 [===========================>..] - ETA: 4s - loss: 0.0615 - mean_iou: 0.8490

3320/3320 [==============================] - 88s 26ms/step - loss: 0.0616 - mean_iou: 0.8491
Epoch 22/50
3320/3320 [==============================] - 88s 26ms/step - loss: 0.0600 - mean_iou: 0.8504
Epoch 23/50
3152/3320 [===========================>..] - ETA: 4s - loss: 0.0589 - mean_iou: 0.8515

3320/3320 [==============================] - 88s 26ms/step - loss: 0.0592 - mean_iou: 0.8515


In [35]:
print("Predicting")
# Predict on test data
test_mask = u_net.predict(test_img,verbose=1)
print(test_mask)

Predicting
65/65 [==============================] - 1s 19ms/step
[[[[9.38344654e-03]
   [2.89355288e-04]
   [7.42524790e-05]
   ...
   [1.21620025e-04]
   [7.72032145e-05]
   [7.29029998e-04]]

  [[1.36758550e-04]
   [1.28072395e-06]
   [2.63106841e-07]
   ...
   [7.84409110e-07]
   [1.80867630e-07]
   [1.07561364e-05]]

  [[3.60798804e-05]
   [1.70375259e-07]
   [3.04862482e-08]
   ...
   [8.66462528e-07]
   [1.17913906e-07]
   [3.03854631e-06]]

  ...

  [[3.88836592e-01]
   [4.19455141e-01]
   [1.52286991e-01]
   ...
   [1.03854882e-06]
   [2.75052344e-07]
   [1.20143332e-05]]

  [[2.47187898e-01]
   [1.55417204e-01]
   [4.06072177e-02]
   ...
   [1.20006814e-06]
   [5.02530156e-07]
   [1.96168530e-05]]

  [[2.00762644e-01]
   [8.79957750e-02]
   [3.18726599e-02]
   ...
   [1.19449156e-04]
   [7.66451340e-05]
   [1.01901055e-03]]]


 [[[3.70975733e-02]
   [4.78941528e-03]
   [4.85911220e-03]
   ...
   [1.22671670e-04]
   [1.61779812e-04]
   [1.80882530e-03]]

  [[2.28828983e-03]
   

In [0]:
# Create list of upsampled test masks
test_mask_upsampled = []
for i in range(len(test_mask)):
    test_mask_upsampled.append(resize(np.squeeze(test_mask[i]),
                                       (test_img_sizes[i][0],test_img_sizes[i][1]), 
                                       mode='constant', preserve_range=True))
    


In [0]:
test_ids, rles = mask_to_rle(test_mask_upsampled)

In [0]:
# Create submission DataFrame
sub = pd.DataFrame()
sub['ImageId'] = test_ids
sub['EncodedPixels'] = pd.Series(rles).apply(lambda x: ' '.join(str(y) for y in x))

sub.to_csv('submission-u-net.csv', index=False)

In [0]:
# Code to download files from Google colab

from google.colab import files

files.download('submission-u-net.csv')